The following four components of connotative meaning (or connotative meanings) are most widely recognized:
1) stylistic - indicating the register, or the situation of the communication;
2) emotive - revealing emotions of the speaker;
3) expressive - aiming at creating the image of the object or intensifying a definite phenomenon;
4) evaluative - stating the value of the indicated event or process.

📌 Что будет делать код

Пользователь вводит текст.

Текст очищается и токенизируется (разбивается на слова).

Каждое слово прогоняется через наши функции анализа.

Результаты складываются в pandas.DataFrame.

Сохраняются в connotation_analysis.csv

In [ ]:
!pip install nltk transformers torch

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

1. Токенизация, удаление стоп-слов, приведение к нижнему регистру

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

sentence = "it's beastly cold today!"
tokens = nltk.word_tokenize(sentence)

# Получаем английские стоп-слова
stop_words = set(stopwords.words('english'))

# Фильтруем токены с приведением к нижнему регистру
filtered_tokens = [w for w in tokens if w.lower() not in stop_words]

print(filtered_tokens)


["'s", 'beastly', 'cold', 'today', '!']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2. Поиск денотата:

Для каждого токена ищем денотативное значение через WordNet.
Складываем результат в табличку (pandas DataFrame) с колонками word и denotation.

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Входной текст
text = "it's beastly cold today!"

# Токенизация
tokens = nltk.word_tokenize(text)

# Поиск денотата
results = []
for w in tokens:
    synsets = wn.synsets(w)
    denotation = synsets[0].definition() if synsets else ""
    results.append({"word": w, "denotation": denotation})

# Создаём таблицу
df = pd.DataFrame(results)
print(df)


      word                                         denotation
0       it  the branch of engineering that deals with the ...
1       's                                                   
2  beastly                                    very unpleasant
3     cold  a mild viral infection involving the nose and ...
4    today                            the present time or age
5        !                                                   


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


3. Emotive connotation

Цифры в скобках — это вероятность (score) модели GoEmotions, с которой она предсказывает данный эмоциональный лейбл для слова.

In [ ]:
import nltk
from transformers import pipeline
import pandas as pd

nltk.download('punkt')

# GoEmotions для эмотивной окраски
emotion_analyzer = pipeline(
    "text-classification",
    model="bhadresh-savani/bert-base-go-emotion",
    return_all_scores=False
)

#  Функция анализа emotive
def get_emotive(word):
    try:
        result = emotion_analyzer(word)[0]
        return f"{result['label']} ({result['score']:.2f})"
    except:
        return "neutral"

#  Токенизация
sentence = "it's fucking cold today!"
tokens = nltk.word_tokenize(sentence)

# Анализ каждого токена
results = []
for w in tokens:
    emotive = get_emotive(w)
    results.append({"word": w, "emotive": emotive})

# Создание таблицы
df = pd.DataFrame(results)
print(df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cuda:0


      word         emotive
0       it  neutral (0.87)
1       's  neutral (0.92)
2  fucking    anger (0.72)
3     cold  neutral (0.95)
4    today  neutral (0.96)
5        !  neutral (0.87)


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


4. Evaluative (nltk  SentimentIntensityAnalyzer)

In [ ]:
import spacy
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Загрузка моделей
nlp = spacy.load("en_core_web_sm")  # spacy для токенизации и лемматизации
nltk.download('vader_lexicon')      # словарь VADER
sia = SentimentIntensityAnalyzer()  # инициализация VADER

# Функция evaluative
def get_evaluative(word):
    """Оценочная коннотация (Positive / Negative / Neutral)"""
    score = sia.polarity_scores(word)['compound']
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    return "Neutral"

#  Основной анализ
def analyze_evaluative(text):
    doc = nlp(text)
    # берём только алфавитные токены
    words = [token.lemma_ for token in doc if token.is_alpha]

    results = []
    for w in words:
        results.append({
            "word": w,
            "evaluative": get_evaluative(w)
        })
    return pd.DataFrame(results)

#  Пример
input_text = " it's beastly cold today fuck!"
df = analyze_evaluative(input_text)
print(df)


      word evaluative
0       it    Neutral
1  beastly    Neutral
2     cold    Neutral
3    today    Neutral
4     fuck   Negative


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


3. Expressive

Используем VADER для оценки evaluative. если слово сильно позитивное или негативное, это уже сигнал экспрессивности.

Используем GoEmotions, если слово имеет высокую уверенность (score) в эмоциональной категории, оно тоже считается экспрессивным.

In [ ]:
import spacy
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline

#  Загрузка моделей
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# GoEmotions
emotion_analyzer = pipeline(
    "text-classification",
    model="bhadresh-savani/bert-base-go-emotion",
    return_all_scores=False
)

#  Функции
def get_evaluative(word):
    score = sia.polarity_scores(word)['compound']
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    return "Neutral"

def get_emotive(word):
    try:
        result = emotion_analyzer(word)[0]
        return f"{result['label']} ({result['score']:.2f})"
    except:
        return "neutral"

def get_expressive(word):
    try:
        label = emotion_analyzer(word)[0]['label'].lower()
        return "Expressive" if label != "neutral" else "Not expressive"
    except:
        return "Not expressive"

# Анализ текста
def analyze_text(text):
    doc = nlp(text)
    words = [token.lemma_ for token in doc if token.is_alpha]

    results = []
    for w in words:
        results.append({
            "word": w,
            "expressive": get_expressive(w)
        })

    return pd.DataFrame(results)

#  input
text = "it's beastly cold today fuck!"
df = analyze_text(text)
print(df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


      word      expressive
0       it  Not expressive
1  beastly  Not expressive
2     cold  Not expressive
3    today  Not expressive
4     fuck      Expressive


4. Stylistic самое изи но самое жесть

Terms
Poetical words
Archaic words
Foreignisms and barbarisms
Literary nonce-words or neologisms
professionalisms
jargonisms
dialectisms
vulgarisms
colloquial nonce-words

[вот](https://docs.yandex.ru/docs/view?tm=1757836414&tld=ru&lang=en&name=LECTURE%202.pdf&text=archaic%20poetic%20slang&url=https%3A%2F%2Fpereklad.nmu.org.ua%2Fua%2FLECTURE%25202.pdf&lr=216&mime=pdf&l10n=ru&sign=2f67dac3c98d2ea649ed4a7d4c3927af&keyno=0&nosw=1&serpParams=tm%3D1757836414%26tld%3Dru%26lang%3Den%26name%3DLECTURE%25202.pdf%26text%3Darchaic%2Bpoetic%2Bslang%26url%3Dhttps%253A%2F%2Fpereklad.nmu.org.ua%2Fua%2FLECTURE%2525202.pdf%26lr%3D216%26mime%3Dpdf%26l10n%3Dru%26sign%3D2f67dac3c98d2ea649ed4a7d4c3927af%26keyno%3D0%26nosw%3D1 )



Модель основана на XLM-Roberta — это трансформер, который умеет работать с многоязычными текстами.

Она обучена на датасете XFORMAL, где тексты размечены как Formal / Informal.

Обучение происходило на предложениях или фрагментах текста, а не на отдельных словах.

То есть модель учится видеть признаки формальности в словосочетаниях и контексте, а не по одному слову.


Этот код токенизирует текст и фильтрует только слова (игнорируя пунктуацию и стоп-слова).
Для каждого слова функция get_stylistic использует модель s-nlp/xlmr_formality_classifier, чтобы классифицировать его как Formal, Informal или Neutral.
Результаты сохраняются в список словарей, который затем преобразуется в таблицу pandas с колонками word и stylistic.
В конце пример показывает применение этой функции к предложению и выводит DataFrame с оценкой стиля каждого слова.

In [ ]:
from transformers import pipeline
import nltk
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Параметры
stop_words = set(stopwords.words('english'))

#модель классификации формальности
classifier = pipeline("text-classification", model="s-nlp/xlmr_formality_classifier")

def get_stylistic(word):
    """Классификация слова по стилю (formal / informal / neutral)"""
    # Если слово стоп-слово => Neutral
    if word.lower() in stop_words:
        return "Neutral"
    # Модель требует предложения, но мы подаём слово само
    try:
        result = classifier(word)[0]
        label = result['label']  # 'FORMAL' или 'INFORMAL', либо Neutral
        if label.upper() == "FORMAL":
            return "Formal"
        elif label.upper() == "INFORMAL":
            return "Informal"
    except:
        return "Neutral"
    return "Neutral"

def analyze_text_stylistic(text):
    tokens = [w for w in word_tokenize(text) if w.isalpha()]
    results = []
    for w in tokens:
        stylistic = get_stylistic(w)
        results.append({"word": w, "stylistic": stylistic})
    return pd.DataFrame(results)

#  Пример
text = "it's fucking cold today and I love the steed"
df = analyze_text_stylistic(text)
print(df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Device set to use cuda:0


      word stylistic
0       it   Neutral
1  fucking  Informal
2     cold  Informal
3    today  Informal
4      and   Neutral
5        I   Neutral
6     love  Informal
7      the   Neutral
8    steed  Informal


Здесь stylistic определяется для всего текста.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Загрузка модели и токенизатора
model_name = "s-nlp/xlmr_formality_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Пример текста
text = "what a beastly weather today!?"

# Токенизация текста
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Получение предсказания
with torch.no_grad():
    logits = model(**inputs).logits

# Определение класса
predicted_class = torch.argmax(logits, dim=1).item()
class_label = "Formal" if predicted_class == 0 else "Informal"

print(f"Predicted class: {class_label}")


Predicted class: Informal


Черновые штуки

In [ ]:
for synset in wn.synsets(word):
  print(f"Syn: {[l.name() for l in synset.lemmas()]}")

Syn: ['fuck', 'fucking', 'screw', 'screwing', 'ass', 'nooky', 'nookie', 'piece_of_ass', 'piece_of_tail', 'roll_in_the_hay', 'shag', 'shtup']
Syn: ['sleep_together', 'roll_in_the_hay', 'love', 'make_out', 'make_love', 'sleep_with', 'get_laid', 'have_sex', 'know', 'do_it', 'be_intimate', 'have_intercourse', 'have_it_away', 'have_it_off', 'screw', 'fuck', 'jazz', 'eff', 'hump', 'lie_with', 'bed', 'have_a_go_at_it', 'bang', 'get_it_on', 'bonk']
Syn: ['bally', 'blinking', 'bloody', 'blooming', 'crashing', 'flaming', 'fucking']
Syn: ['fucking']


In [ ]:
## evaluative, emotive, expressive, stylistic

1. Sentiment Analysis (тональность / polarity detection)

For evaluative connotation: positive / negative.

Библиотеки:

NLTK (VADER Sentiment Analyzer) - ok

TextBlob

Hugging Face модели (bert-base-uncased-sentiment, cardiffnlp/twitter-roberta-base-sentiment и др.).

2. Emotion Detection

Модели, которые классифицируют текст по эмоциям (joy, anger, sadness, fear и т.п.).

Используется для эмоциональной коннотации (emotive).

Библиотеки / модели:

GoEmotions (Google, 27 классов эмоций, есть на Hugging Face).

Ekman emotions datasets + классификаторы на BERT.

3. Stylistic Analysis

Для стилистической коннотации можно применять модели register/style detection: archaic, col, sland, etc

https://pypi.org/project/wiktextract/

4. Expressiveness / Intensifiers

Чаще решается через выявление усилителей (intensifiers: very, extremely,
terribly. downtowners: a bit, a sort of, etc) и сравнительный анализ с нейтральными синонимами.

Можно использовать distributional semantics (word embeddings) + корпусные методы (частотность в эмоционально окрашенных контекстах).

Определение иронии

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-irony")

config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-irony")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-irony")

In [ ]:
from transformers import pipeline

# Инициализация пайплайна
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-irony")

# Ваше предложение
sentence = "I love you!"

# Получение результата
result = pipe(sentence)[0]

print(result)


Device set to use cuda:0


{'label': 'non_irony', 'score': 0.5125525593757629}


In [ ]:
from transformers import pipeline

# Инициализация пайплайна
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-irony")

# Ваше предложение
sentence = "Oh great, another Monday!"

# Получение результата
result = pipe(sentence)[0]

print(result)


Device set to use cuda:0


{'label': 'irony', 'score': 0.9874765276908875}
